# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#подключим необходимые для работы библиотеки

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
pd.options.display.float_format = '{:,.2f}'.format

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [2]:
#прочитаем исходный датафрейм

try:
    data = pd.read_csv('C:/Users/jnybfuvybtfgl kfolbl/mypythonworks/yandex/users_behavior.csv', sep=',') 
    
except:
    data = pd.read_csv('/datasets/users_behavior.csv', sep=',')

In [3]:
# изучим основную информацию о датафрейме
print('Первые 10 строк датафрейма:')
display(data.head(10))
print('Основная информация о датафрейме:')
display(data.info())
print('Размер датасета:')
display(data.shape)

Первые 10 строк датафрейма:


,calls,minutes,messages,mb_used,is_ultra
0,40.00,311.90,83.00,"19,915.42",0
1,85.00,516.75,56.00,"22,696.96",0
2,77.00,467.66,86.00,"21,060.45",0
3,106.00,745.53,81.00,"8,437.39",1
4,66.00,418.74,1.00,"14,502.75",0
5,58.00,344.56,21.00,"15,823.37",0
6,57.00,431.64,20.00,"3,738.90",1
7,15.00,132.40,6.00,"21,911.60",0
8,7.00,43.39,3.00,"2,538.67",1
9,90.00,665.41,38.00,"17,358.61",0


Основная информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

Размер датасета:


(3214, 5)

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Изучив исходные данные датафрейма можно сказать, что данные представлены в полном объеме, типы данных корректны. Также в представленном датафрейме уже имеется целевой признак - "is_ultra".
</div>

## Разбейте данные на выборки

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Т.к. у нас нет спрятанной тестовой выборки поделим исходные данные на три части. Сделаем размеры валидационного и тестового наборов одинаковыми, а общее соотношение обучающего, валидационного и тестового наборов примем 3:1:1 соответственно, т.е. 60%:20%:20%. 
</div>

In [4]:
#для начала отделим обучающую выборку от валидационной и тестовой

df_train, df_valid_test = train_test_split(data, test_size=0.4, random_state=12345)

#далее отделим тестовую выборку от валидационной

df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)

print('Размер обучающей выборки равен: ', df_train.shape)
print('Размер валидационной выборки равен: ', df_valid.shape)
print('Размер тестовой выборки равен: ', df_test.shape)

Размер обучающей выборки равен:  (1928, 5)
Размер валидационной выборки равен:  (643, 5)
Размер тестовой выборки равен:  (643, 5)


## Исследуйте модели

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Перейдем к исследованию различных моделей, таких как дерево решений, случайных лес и логистическая регрессия. Меняя гиперпараметры, определим какая модель лучше себя покажет на валидационной выборке.
</div>

### Изучение модели "Дерево решений"

In [5]:
#подготовим переменные с признаками и целевым признаком для обучающей и валидационной выборок

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid =  df_valid.drop(['is_ultra'], axis=1)
target_valid =  df_valid['is_ultra']

In [6]:
#самый важный гиперпараметр у решающего дерева это глубина решающего дерева
#создадим цикл для перебора глубины дерева от 1 до 50
#и определим при какой гулбине на валидационной выборке лучшее качество модели

best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 50):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print('Максимальное качество на валидационной выборке достигается при глубине дерева равном:', best_depth)
print('Accuracy наилучшей модели на валидационной выборке:', best_result)

Максимальное качество на валидационной выборке достигается при глубине дерева равном: 3
Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


### Изучение модели "Случайный лес"

In [7]:
#в алгоритме классификациии "Случайный лес" важным гиперпараметром является количество решающих деверьев
#создадим цикл для перебора количества дереьев от 1 до 50
#и определим при каком количестве деревьев на валидационной выборке лучшее качество модели

best_model = None
best_result = 0
best_est = 0
min_samples_split = 2
min_samples_leaf = 1

for est in range(1, 50):
    for split in range(2, 6):
        for leaf in range(1, 5):
            model = RandomForestClassifier(random_state=12345, n_estimators=est, min_samples_split=split, min_samples_leaf=leaf) 
            model.fit(features_train, target_train)
            result = model.score(features_valid, target_valid)
            if result > best_result:
                best_model = model
                best_result = result
                best_est = est
                min_samples_split = split
                min_samples_leaf = leaf
                
print('Максимальное качество на валидационной выборке достигается при минимальном количестве примеров для разделения равном:', split)                
print('Максимальное качество на валидационной выборке достигается при минимальном количестве объектов в листе:', leaf)        
print('Максимальное качество на валидационной выборке достигается при количестве деревьев равном:', best_est)
print('Accuracy наилучшей модели на валидационной выборке:', best_result)

Максимальное качество на валидационной выборке достигается при минимальном количестве примеров для разделения равном: 5
Максимальное качество на валидационной выборке достигается при минимальном количестве объектов в листе: 4
Максимальное качество на валидационной выборке достигается при количестве деревьев равном: 23
Accuracy наилучшей модели на валидационной выборке: 0.8009331259720062


### Изучение модели "Логистическая регрессия"

In [8]:
#в алгоритме "Логистическая регрессия" важными гиперпараметрами являются алгоритм, 
#который будет строить модель и количество итераций обучения
#создадим цикл для перебора количества итераций от 1 до 50
#и определим при каком количестве итераций на валидационной выборке лучшее качество модели

best_model = None
best_result = 0
best_iter = 0
for iters in range(1, 50):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iters)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_iter = iters

print('Максимальное качество на валидационной выборке достигается при количестве итераций равном:', best_iter)
print('Accuracy наилучшей модели на валидационной выборке:', best_result)

Максимальное качество на валидационной выборке достигается при количестве итераций равном: 39
Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715


### Вывод

Нами были проверены три модели машинного обучения и получены следующие результаты:
- "Дерево решений": наилучшее качество модели 0.7853810264385692 достигается при глубине дерева равном 3 
- "Случайный лес": наилучшее качество модели 0.8009331259720062 достигается при количестве деревьев равном 23, при минимальном количестве объектов в листе равном 4 и минимальном количестве примеров для разделения равном 5
- "Логистическая регрессия": наилучшее качество модели 0.7107309486780715 достигается при количестве итераций равном 39

Для проверки качества модели на тестовой выборке применим модель "Случайный лес", т.к. она показала наилучший результат на валидационной выборке.

## Проверьте модель на тестовой выборке

In [9]:
#подготовим переменные с признаками и целевым признаком для тестовой выборки

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [10]:
#применим нашу лучшую модель для предсказания ответов на тестовой выборке

model = RandomForestClassifier(random_state=12345, n_estimators=23, min_samples_split=5, min_samples_leaf=4) 
model.fit(features_train, target_train)
result = model.score(features_test, target_test)

print('Accuracy модели на тестовой выборке:', result)

Accuracy модели на тестовой выборке: 0.80248833592535


## (бонус) Проверьте модели на адекватность

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Не совсем понял что необходимо сделать в данном разделе. Попытался сравнить свою модель и стандартную модель для классификации.
Уровень на обученной модели получился выше, значит модель вполне работоспособна.
</div>

In [11]:
from sklearn.dummy import DummyClassifier

model = DummyClassifier(random_state=12345) 
model.fit(features_train, target_train)
result = model.score(features_test, target_test)

print('Accuracy модели на тестовой выборке:', result)

Accuracy модели на тестовой выборке: 0.6842923794712286
